In [1]:
# Set working directory
import os
os.chdir('C:/Users/sm2511/PycharmProjects/TLOmodel')

In [2]:
from scripts.consumable_resource_analyses.resource_functions import *
import datetime
import pandas as pd
import shapefile as shp
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from pathlib import Path
import re

# define a timestamp for script outputs
timestamp = datetime.datetime.now().strftime("_%Y_%m_%d_%H_%M")

# print the start time of the script
print('Script Start', datetime.datetime.now().strftime('%H:%M'))

# define a pathway to the data folder (note: currently outside the TLO model directory)
# remember to set working directory to TLOmodel/
outputfilepath = Path("./outputs")
resourcefilepath = Path("./resources")
datafilepath = Path("./../../Documents/health_systems/data")

Script Start 12:17


In [3]:
# ----------------------------------------------------------------------------------------------------------------------
# active analysis script
# ----------------------------------------------------------------------------------------------------------------------

#%% Data import and cleaning

# importing 2018 LMIS data
df_import = pd.read_csv(resourcefilepath / 'ResourceFile_LMIS_2018.csv', low_memory = False)
#df_import = pd.read_csv('C:/Users/sm2511/PycharmProjects/TLOmodel/resources/ResourceFile_LMIS_2018.csv', low_memory=False)

# importing facilities data
master_facilities_df = pd.read_csv(resourcefilepath / 'ResourceFile_Master_Facilities_List.csv')
master_facilities_df = master_facilities_df.drop(columns=['Unnamed: 0'])
master_facilylist_df = pd.read_csv(resourcefilepath / 'ResourceFile_Facilities_For_Each_District.csv')

# importing consumables list (consumables called upon by the disease modules)
consumables_df = pd.read_csv(resourcefilepath / 'ResourceFile_Consumables_new.csv')
consumables_df = consumables_df.drop(columns=['Unnamed: 0'])
#consumables_df.columns = consumables_df.iloc[0]
consumables_df = consumables_df.drop(consumables_df.index[0:4])
consumables_df = consumables_df.reindex()
consumables_df = consumables_df.dropna( how='any',
                    subset=['Item_Code Description'])

#nonalphanum_names = consumables_df[(consumables_df['Item_Code Used'] == "78")|(consumables_df['Item_Code Used'] == "1902")].index
#consumables_df.drop(nonalphanum_names, inplace = True)

consumables_df["Item_Code Description"] = consumables_df["Item_Code Description"].str.replace("\'", "").str.strip()
print('Completed data import')

# extract unique facilities and consumables from the resource files
#facility_names_list = master_facilities_df['Facility Name'].unique()
consumables_list = consumables_df['Item_Code Description_clean'].unique()
df_resources_list = df_import['Fullproductname'].unique()
df_facilities_list = df_import['Name'].unique()

Completed data import


In [4]:
print(len(df_resources_list), "unique resources in the LMIS database\n",
     len(consumables_list), "unique resources in the consumables database")

393 unique resources in the LMIS database
 212 unique resources in the consumables database


## Step 2 
Match the names of resources in the Consumables database to the resource names in the LMIS dataset
1. Try fuzzy matching using various matching mathods - simple ratio, partial ratio, token sort ratio, token set ratio
2. Keep good matches from the fuzzy matching process (score > 90)
3. Run manual checks on "partial matches" (score < 90)
4. Manual matching for unmatched resources

In [5]:
# Fuzzy matching consumable resources, using various methods
print('Fuzzy matching consumable resources using simple ratio')
matches = fuzzy_finder_function(df_resources_list, consumables_list, fuzz.ratio)
resource_matches = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
resource_matches['Fullproductname'] = df_resources_list
resource_matches_simpr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

print('Fuzzy matching consumable resources using partial ratio')
matches = fuzzy_finder_function(df_resources_list, consumables_list, fuzz.partial_ratio)
resource_matches = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
resource_matches['Fullproductname'] = df_resources_list
resource_matches_pr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

print('Fuzzy matching consumable resources using token set ratio')
matches = fuzzy_finder_function(df_resources_list, consumables_list, fuzz.token_set_ratio)
resource_matches = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
resource_matches['Fullproductname'] = df_resources_list
resource_matches_tsetr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

print('Fuzzy matching consumable resources using token sort ratio')
matches = fuzzy_finder_function(df_resources_list, consumables_list, fuzz.token_sort_ratio)
resource_matches = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
resource_matches['Fullproductname'] = df_resources_list
resource_matches_tsortr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

Fuzzy matching consumable resources using simple ratio
Fuzzy matching consumable resources using partial ratio
Fuzzy matching consumable resources using token set ratio
Fuzzy matching consumable resources using token sort ratio


In [6]:
# Iteratively append good matches to a dataframe
# 1. Match based on simple ratio
resource_match_finalsimp = resource_matches_simpr[resource_matches_simpr['Confidence'] > 90]
resource_match_final = resource_match_finalsimp
resource_match_final['MatchingMethod'] = 'Simple Ratio'

# 2a. Match based on partial ratio
resource_match_finalpr = resource_matches_pr[resource_matches_pr['Confidence'] > 90]

# 2b. Append matches based on partial ratio to the final matched dataframe
merge = pd.merge(resource_match_final, resource_match_finalpr, on = 'ResourceFile_Match', how='outer')
resource_match_finalpr = merge.drop_duplicates()
c1 = resource_match_finalpr['Confidence_x'] < 90
c2 = resource_match_finalpr['Confidence_x'].notna()
resource_match_finalpr = resource_match_finalpr[~c2]

resource_match_finalpr = resource_match_finalpr.drop(['Confidence_x', 'Fullproductname_x'], axis=1)
resource_match_finalpr =resource_match_finalpr.rename(columns={"Confidence_y":"Confidence", "Fullproductname_y": "Fullproductname"})
resource_match_finalpr['MatchingMethod'] = 'Partial Ratio'
resource_match_final = resource_match_final.append(resource_match_finalpr)

# 3a. Match based on token sort ratio
resource_match_finaltsortr = resource_matches_tsortr[resource_matches_tsortr['Confidence'] > 90]

# 3b. Append matches based on token sort ratio to the final matched dataframe
merge = pd.merge(resource_match_final, resource_match_finaltsortr, on = 'ResourceFile_Match', how='outer')
resource_match_finaltsortr = merge.drop_duplicates()
c1 = resource_match_finaltsortr['Confidence_x'] < 90
c2 = resource_match_finaltsortr['Confidence_x'].notna()
resource_match_finaltsortr = resource_match_finaltsortr[~c2]

resource_match_finaltsortr = resource_match_finaltsortr.drop(['Confidence_x', 'Fullproductname_x'], axis=1)
resource_match_finaltsortr =resource_match_finaltsortr.rename(columns={"Confidence_y":"Confidence", "Fullproductname_y": "Fullproductname"})
resource_match_finaltsortr['MatchingMethod'] = 'Token Sort Ratio'
resource_match_final = resource_match_final.append(resource_match_finaltsortr)

# 4a. Match based on token set ratio
resource_match_finaltsetr = resource_matches_tsetr[resource_matches_tsetr['Confidence'] > 90]

# 4b. Append matches based on token set ratio to the final matched dataframe
merge = pd.merge(resource_match_final, resource_match_finaltsetr, on = 'ResourceFile_Match', how='outer')
resource_match_finaltsetr = merge.drop_duplicates()
c1 = resource_match_finaltsetr['Confidence_x'] < 90
c2 = resource_match_finaltsetr['Confidence_x'].notna()
resource_match_finaltsetr = resource_match_finaltsetr[~c2]

resource_match_finaltsetr = resource_match_finaltsetr.drop(['Confidence_x', 'Fullproductname_x'], axis=1)
resource_match_finaltsetr =resource_match_finaltsetr.rename(columns={"Confidence_y":"Confidence", "Fullproductname_y": "Fullproductname"})
resource_match_finaltsetr['MatchingMethod'] = 'Token Set Ratio'
resource_match_final = resource_match_final.append(resource_match_finaltsetr)

# Final checks and extract matched dataframe
assert len(resource_match_final) == len(resource_match_finalpr) + len(resource_match_finalsimp) + len(resource_match_finaltsortr) +  + len(resource_match_finaltsetr)
print(len(resource_match_final['ResourceFile_Match'].unique()), "consumables matched out of ", len(consumables_list), "from", len(df_resources_list), " in the 2018 LMIS data")
resource_match_final = resource_match_final.drop_duplicates(subset = ['ResourceFile_Match','Fullproductname'])
resource_match_final.to_csv('C:/Users/sm2511/OneDrive - University of York/Documents/TLO-local/resource_match.csv')

44 consumables matched out of  212 from 393  in the 2018 LMIS data


<ipython-input-6-071c79ef64b3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_match_final['MatchingMethod'] = 'Simple Ratio'


In [8]:
# Merge the two matched lists and extract into an excel file for manual manipulation
resource_partmatch = resource_matches_simpr
resource_partmatch['MatchingMethod'] = 'Simple Ratio'

resource_partmatch = resource_partmatch.append(resource_matches_pr)
c1 = resource_partmatch['MatchingMethod'].notna()
resource_partmatch.loc[~c1, 'MatchingMethod'] = 'Partial Ratio'

resource_partmatch = resource_partmatch.append(resource_matches_tsortr)
c1 = resource_partmatch['MatchingMethod'].notna()
resource_partmatch.loc[~c1, 'MatchingMethod'] = 'Token Sort Ratio'

resource_partmatch = resource_partmatch.append(resource_matches_tsetr)
c1 = resource_partmatch['MatchingMethod'].notna()
resource_partmatch.loc[~c1, 'MatchingMethod'] = 'Token Set Ratio'

resource_partmatch = resource_partmatch.dropna(how='all')
resource_partmatch = resource_partmatch.drop_duplicates()

resource_partmatch = pd.merge(resource_partmatch, resource_match_final, on = 'ResourceFile_Match', how = 'outer')
c1 = resource_partmatch['Confidence_y'].notna()
resource_partmatch = resource_partmatch[~c1]

resource_partmatch = resource_partmatch.drop(['Confidence_y','Fullproductname_y','MatchingMethod_y'],axis = 1)
resource_partmatch = resource_partmatch.rename(columns={"Confidence_x":"Confidence", "Fullproductname_x": "Fullproductname", 
                                                        "MatchingMethod_x":"MatchingMethod"})

resource_partmatch = resource_partmatch.drop_duplicates(subset = ['ResourceFile_Match','Fullproductname'])

resource_partmatch.to_csv('C:/Users/sm2511/OneDrive - University of York/Documents/TLO-local/resource_unmatched.csv')

In [ ]:
consumable_names_list = consumables_df['Items'].unique()
consumable_names_list = pd.DataFrame(consumable_names_list, columns=['ResourceFile_Match'])
resource_unmatched = pd.merge(consumable_names_list, resource_partmatch, on = 'ResourceFile_Match', how = 'outer')
resource_unmatched = resource_unmatched.dropna(how='all')
c1 = resource_unmatched['Fullproductname'].notna()
resource_unmatched = resource_unmatched[~c1]

In [ ]:
all_consumables = resource_partmatch.append(resource_unmatched)
all_consumables.to_csv('C:/Users/sm2511/OneDrive - University of York/Documents/TLO-local/resource_partmatched_and_unmatched.csv')

In [ ]:
# Fuzzy match consumables names list with itself
consumable_names_list = consumables_df['Items'].unique()
print('Fuzzy matching consumable resources using partial ratio')
matches = fuzzy_finder_function(consumable_names_list, consumable_names_list, fuzz.partial_ratio)
consumables_cleaned = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
#resource_matches['Fullproductname'] = df_resources_list
#resource_matches_simpr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

In [ ]:
# Fuzzy match consumables names list with itself
consumable_names_list = consumables_df['Items'].unique()
print('Fuzzy matching consumable resources using partial ratio')
matches = fuzzy_finder_function(consumable_names_list, consumable_names_list, fuzz.partial_ratio)
consumables_cleaned = pd.DataFrame(matches, columns=['ResourceFile_Match', 'Confidence'])
#resource_matches['Fullproductname'] = df_resources_list
#resource_matches_simpr = resource_matches.reset_index(drop=True).sort_values(by='Confidence', ascending=False)

In [10]:
consumables_cleaned['otherpossible_drugname'] = consumable_names_list
consumables_cleaned = consumables_cleaned.reset_index(drop=True).sort_values(by='Confidence', ascending=False)
consumables_cleaned

NameError: name 'consumable_names_list' is not defined